In [1]:
import os
import json
from pathlib import Path

from sacrebleu import sentence_bleu
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

2021-07-20 12:25:17.989330: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


## Continuation generation

In [3]:
def generate_paragraph(path_model: str, path_tokenizer: str, index_news: int, path_log: str) -> None:
    path_dataset_test = '../../dataset/digi24/split/test.json'
    tokenizer = GPT2Tokenizer.from_pretrained(path_tokenizer)
    model = TFGPT2LMHeadModel.from_pretrained(path_model)

    with open(path_dataset_test, 'r') as input_file:
        data = json.load(input_file)

    if index_news > len(data) - 1:
        print(f'Error, max index is: {len(data) - 1}')
        return

    example = data[index_news]
    title = example['title']
    text = ' '.join(example['text'][:-1])
    original_text = example['text'][-1]
    inputs_text = f'Text: {title} {text} Continuation:'

    tokens_input = tokenizer.encode(inputs_text, return_tensors='tf')
    tokens_predict = model.generate(tokens_input, max_length=1024, no_repeat_ngram_size=2)[0][len(tokens_input[0]):]
    predict_text = tokenizer.decode(tokens_predict).replace('<|endoftext|>', '')

    Path(os.path.dirname(path_log)).mkdir(parents=True, exist_ok=True)
    with open(path_log, 'w+') as output_file:
        output_file.write(f"Input:\t{inputs_text}\n")
        output_file.write(f"Predict:\t{predict_text}\n")
        output_file.write(f"Original:\t{original_text}\n")

    print(f"Input:\t{inputs_text}\n")
    print(f"Predict:\t{predict_text}\n")
    print(f"Original:\t{original_text}\n")

    bleu = sentence_bleu(predict_text, [original_text]).score
    print(f'Bleu score: {bleu}\n')

## Title generation

In [5]:
def generate_title(path_model: str, path_tokenizer: str, index_news: int, path_log: str) -> None:
    path_dataset_test = '../../dataset/digi24/split/test.json'
    tokenizer = GPT2Tokenizer.from_pretrained(path_tokenizer)
    model = TFGPT2LMHeadModel.from_pretrained(path_model)

    with open(path_dataset_test, 'r') as input_file:
        data = json.load(input_file)

    example = data[index_news]
    original_title = example['title']
    text = ' '.join(example['text'])

    inputs_text = f'Text: {text} Title: '
    inputs_token = tokenizer.encode(inputs_text, return_tensors='tf')
    text_predict = model.generate(inputs_token, max_length=1024, no_repeat_ngram_size=2)[0][len(inputs_token[0]):]
    predict_text = tokenizer.decode(text_predict).replace('<|endoftext|>', '')


    Path(os.path.dirname(path_log)).mkdir(parents=True, exist_ok=True)
    with open(path_log, 'w+') as output_file:
        output_file.write(f"Input:\t{inputs_text}\n")
        output_file.write(f"Predict:\t{predict_text}\n")
        output_file.write(f"Original:\t{original_title}\n")

    print(f"Input:\t{inputs_text}\n")
    print(f"Predict:\t{predict_text}\n")
    print(f"Original:\t{original_title}\n")

    bleu = sentence_bleu(predict_text, [original_title]).score
    print(f'Bleu score: {bleu}\n')

Example

In [7]:
index_news = 6788

## Continuation generation

In [8]:
generate_paragraph('../../model/news/paragraph/large', '../../model/tokenizer', index_news, f'../../log/news/{index_news}-news.txt')

2021-07-20 12:27:55.545315: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-20 12:27:55.546745: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-20 12:27:55.582314: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2021-07-20 12:27:55.582350: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-20 12:27:55.584338: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-07-20 12:27:55.584391: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-07

Input:	Text: SUA amenință Rusia: Dacă nu vă distrugeți rachetele de croazieră conform Tratatului, vom încerca să le distrugem noi Rusia trebuie să oprească dezvoltarea sistemului de rachete de croazieră care încalcă Tratatul nuclear semnat la sfârşitul Războiului Rece, iar dacă nu se va întâmpla acest lucru Statele Unite vor încerca să distrugă rachetele înainte să devină funcţionale, a afirmat Kay Bailey Hutchison, ambasadoarea SUA la NATO, scrie Reuters. SUA consideră că Rusia dezvoltă un sistem de rachete sol-aer care încalcă tratatul de pe vremea Războiului Rece, Moscova negând însă că ar încălca tratatul. Kay Bailey Hutchison, ambasadoarea SUA la NATO, a declarat că Washingtonul doreşte în continuare o soluţie diplomatică, dar SUA sunt pregătite să ia în considerare un atac militar dacă Rusia va continua să dezvolte sistemul de rachete. „Dacă se va ajunge în acel punct, vom analiza posibilitatea să distrugem o rachetă rusească care ar putea lovi una dintre ţările noastre”, a decla

## Title generation

In [9]:
generate_title('../../model/news/title/medium', '../../model/tokenizer', index_news, f'../../log/news/{index_news}-title.txt')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ../../model/news/title/medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to 0 (first `eos_token_id`) to generate sequence


Input:	Text: Rusia trebuie să oprească dezvoltarea sistemului de rachete de croazieră care încalcă Tratatul nuclear semnat la sfârşitul Războiului Rece, iar dacă nu se va întâmpla acest lucru Statele Unite vor încerca să distrugă rachetele înainte să devină funcţionale, a afirmat Kay Bailey Hutchison, ambasadoarea SUA la NATO, scrie Reuters. SUA consideră că Rusia dezvoltă un sistem de rachete sol-aer care încalcă tratatul de pe vremea Războiului Rece, Moscova negând însă că ar încălca tratatul. Kay Bailey Hutchison, ambasadoarea SUA la NATO, a declarat că Washingtonul doreşte în continuare o soluţie diplomatică, dar SUA sunt pregătite să ia în considerare un atac militar dacă Rusia va continua să dezvolte sistemul de rachete. „Dacă se va ajunge în acel punct, vom analiza posibilitatea să distrugem o rachetă rusească care ar putea lovi una dintre ţările noastre”, a declarat Hutchison. Tratatul Forţelor Nucleare Intermediare (INF), semnat de către preşedintele american Ronald Reagan şi 